In [3]:
import duckdb
from minio import Minio

# Connect to MinIO
minio_client = Minio(
    "localhost:9000",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False
)

# Configure DuckDB for S3 access
conn = duckdb.connect()
conn.execute("""
        INSTALL httpfs;
        LOAD httpfs;
        SET s3_endpoint='minio:9000';
        SET s3_access_key_id='minioadmin';
        SET s3_secret_access_key='minioadmin';
        SET s3_use_ssl=false;
        SET s3_url_style='path';
""")

# Query data from MinIO
result = conn.execute("""
    select country, state, count(distinct(brewery_type)) as unique_types
        from read_parquet(
            's3a://blake/silver/breweries/**/*.parquet',
            hive_partitioning=1
        )
        group by country, state;
""").fetchdf()

print(result)

          country            state  unique_types
0   united_states         colorado             8
1   united_states       new_jersey             6
2   united_states            idaho             6
3   united_states          montana             5
4     south_korea  gyeongsangbukdo             2
..            ...              ...           ...
92    south_korea          daejeon             1
93        ireland        waterford             1
94         france  bouche_du_rhône             1
95        ireland           offaly             1
96        ireland          wexford             1

[97 rows x 3 columns]


In [5]:
result = conn.execute("""
    select *
        from read_parquet(
            's3a://blake/silver/breweries/**/*.parquet',
            hive_partitioning=1
        )
""").fetchdf()

print(result)

                      address_1 address_2 address_3 brewery_type  \
0              Schleppe-Platz 1      None      None        large   
1               Brauhausgasse 6      None      None        large   
2                         L67 7      None      None         nano   
3     Mautner-Markhof-Stra�e 11      None      None        large   
4           Dr.-Beurle-Stra�e 1      None      None        large   
...                         ...       ...       ...          ...   
8350                       None      None      None        micro   
8351            312 Broadway St      None      None        micro   
8352                2549 Hwy 16      None      None        micro   
8353            528 Broadway St      None      None      brewpub   
8354               1930 Main St      None      None      brewpub   

                                        id            latitude  \
0     c88759b6-88d6-4172-bed7-61dec61e50aa    46.6416966591789   
1     f51b3f3d-be14-40da-914a-3b5c83f75bbf   46.616